In [91]:
import json
import pandas as pd
import urllib
import time
import datetime
import re
from bs4 import BeautifulSoup

# Base URL for power data analysis. Use URL with cache3 to speed up scraping
# and not hitting the server too hard. Use following 2 commented lines to
# get freshest data (very slow).
# base_url = "https://www.zwiftpower.com/api3.php?do=analysis_list&zwift_id="
# base_url_appendix = ""

base_url = "https://www.zwiftpower.com/cache3/profile/"
base_url_appendix = "_analysis_list.json"

# top riders JSON is situated here ↓
# https://www.zwiftpower.com/cache3/lists/2_standings_.json .
# Unomment the lines below to use this list of top ~1000 riders
#with urllib.request.urlopen("https://www.zwiftpower.com/"\
#                            "cache3/lists/2_standings_.json") as response:
#    raw_json_top = response.read()
    
# all riders JSON is located here ↓
# https://www.zwiftpower.com/cache3/global/rider_list.json
# A version of the list from 2021-03-17 with most of the JSON attributes
# stripped (only inlcuding name, zwift id, ftp and rank) is available in
# the github repo and can be used by uncommenting the next lines
with open("minified_rider_list.json") as file:
    raw_json_top = file.read()

json_top_rankings_zp = json.loads(raw_json_top)

# We are interested in the the top zwifters personal data (name and ID)
top_zwifter = json_top_rankings_zp.get("data")
top_zwids, riders = [], []

# make lists of zwift_ids and rider names
for zwifter in top_zwifter:
    
    # we only want to check riders with zwiftpower rank less than 500
    if int(zwifter.get("rank")[:-3]) < 500:
        top_zwids.append(zwifter.get("zwid"))
        riders.append(zwifter.get("name"))
    
zwift_ids, setids, names, dates, titles, device_pm, device_st, \
power300_pm, power300_st, group  = [], [], [], [], [], [], [], [], [], []

progress = 0
time_start, time_request = time.time(), 0


print(f"Selected {len(top_zwids)} Zwifters with rank below 500 from"\
      f" {len(top_zwifter)} Zwifters total.")

In [151]:

# iterate over all zwifters, starting from index = progress (default=0)
for zwifter, rider in zip(top_zwids[progress:], riders[progress:]):

    # calculate remaining time
    time_rem = datetime.timedelta(seconds=((time.time() - time_start) / \
                         ((progress+1) / len(top_zwids))))
    # print progress
    print(f"# {progress:4}. ID: {zwifter:8}."\
          f" {progress/len(top_zwids) * 100:5.2f} % finished."\
          f" Request time: {time_request:.2f} s. Est."\
          f" {time_rem} remaining.")

    
    progress += 1
    
    # wait for some seconds – don't flood the server
    #time.sleep(0.5)
    
    time_request_start = time.time()
    # request analysis JSON for rider
    
    
    request = urllib.request.Request(base_url + str(zwifter) + base_url_appendix)
    
    # try to open the URL. Skip to next rider if there is an error
    try:
        response = urllib.request.urlopen(request)
    except urllib.error.HTTPError as e:
        print(e.code)
        print(e.read()) 
        continue
    except urllib.error.URLError as e:
        continue
    
    analysis_json = response.read()
    analysis_json_obj = json.loads(analysis_json)
    time_request_end = time.time()
    time_request = time_request_end - time_request_start

    # get every race for current rider
    races = analysis_json_obj.get("data")
          
    
    # iterate over each race
    for race in races:
        
        # if the race has exactly 2 power values for 300s power...
        # ... and none of the values are zero
        if race.get("power300") and 0 not in race.get("power300"):
            num_devices = len(race.get("power300"))
        else:
            num_devices = 0
        if num_devices == 2:
            devices = []
            
            # device names are stored as name1, name2 etc.
            for i in range(num_devices):
                devices.append(race.get("name" + str(i+1)))
            
            
            if 0 in devices:
                continue

            
            # regex matching kickr but not if bike or snap is included in name
            wahoo_search = [i for i in devices if \
                            (re.search(".*kickr*", i, re.I) and not \
                             re.search(".*bike.*|.snap.*", i, re.I))]
            
            # if match for kickr and 2 devices and max 10 % difference between
            # device power readings, add values to lists.
            if wahoo_search and num_devices == 2 and \
                abs(race.get("power300")[0]/race.get("power300")[1] - 1) \
                < 0.1:
                wahoo_index = devices.index(wahoo_search[0])
                setids.append(race.get("set_id"))
                zwift_ids.append(zwifter)
                names.append(rider)
                dates.append(race.get("date"))
                titles.append(race.get("title"))
                group.append("wahoo")
                if wahoo_index == 1:
                    device_pm.append(devices[0])
                    device_st.append(devices[1])
                    power300_pm.append(race.get("power300")[0])
                    power300_st.append(race.get("power300")[1])
                else:
                    device_pm.append(devices[1])
                    device_st.append(devices[0])
                    power300_pm.append(race.get("power300")[1])
                    power300_st.append(race.get("power300")[0])
                                                   
            
            # regex matching neo but not if 2 comes after neo (except when the)
            # 2 is followed by a 0 (as in 20), so that a Tacx Neo 2017 is 
            # included in the match, but not Tacx Neo 2T
            neo_search = [i for i in devices if \
                          (re.search(".*neo(?![2t])", i, re.I) and not\
                           re.search("2(?!0)", i, re.I))]
            
            if neo_search and num_devices == 2 and \
                abs(race.get("power300")[0]/race.get("power300")[1] - 1) \
                < 0.1:
                neo_index = devices.index(neo_search[0])
                setids.append(race.get("set_id"))
                zwift_ids.append(zwifter)
                names.append(rider)
                dates.append(race.get("date"))
                titles.append(race.get("title"))
                group.append("neo")
                if neo_index == 1:
                    device_pm.append(devices[0])
                    device_st.append(devices[1])
                    power300_pm.append(race.get("power300")[0])
                    power300_st.append(race.get("power300")[1])
                else:
                    device_pm.append(devices[1])
                    device_st.append(devices[0])
                    power300_pm.append(race.get("power300")[1])
                    power300_st.append(race.get("power300")[0])

print(f"Finished! Saved {len(zwift_ids)} rows in {time.time()-time_start} s.")

# 17843. ID:   660565. 45.44 % finished. Request time: 0.90 s. Est. 13:58:51.341678 remaining.
# 17844. ID:   956545. 45.45 % finished. Request time: 0.56 s. Est. 13:58:49.747863 remaining.
# 17845. ID:  1508314. 45.45 % finished. Request time: 0.52 s. Est. 13:58:48.081720 remaining.
# 17846. ID:   915984. 45.45 % finished. Request time: 0.50 s. Est. 13:58:46.371940 remaining.
# 17847. ID:  1475377. 45.45 % finished. Request time: 0.61 s. Est. 13:58:44.900477 remaining.
# 17848. ID:  1545123. 45.46 % finished. Request time: 0.62 s. Est. 13:58:43.435646 remaining.
# 17849. ID:  1532391. 45.46 % finished. Request time: 0.61 s. Est. 13:58:41.957866 remaining.
# 17850. ID:  1504252. 45.46 % finished. Request time: 0.51 s. Est. 13:58:40.265438 remaining.
# 17851. ID:  1529571. 45.46 % finished. Request time: 0.51 s. Est. 13:58:38.576748 remaining.
# 17852. ID:  1473436. 45.47 % finished. Request time: 0.51 s. Est. 13:58:36.883668 remaining.
# 17853. ID:  1375776. 45.47 % finished. Request t

KeyboardInterrupt: 

In [153]:
# mash all of the lists into a much more efficient pandas dataframe structure
df = pd.DataFrame({"zwift_id": zwift_ids, "name": names, "time":\
                   pd.to_datetime(dates, unit="s"), \
                   "group": group, "title": titles, "pm": device_pm,\
                   "st": device_st, "p300s_pm": power300_pm,\
                   "p300s_st": power300_st}, index=setids)

# drop the duplicate values. no mercy!
df = df[~df.duplicated(keep="first")]

# calculate the delta (diffrence) between power meter and smart trainer
df["delta"] = df["p300s_pm"] - df["p300s_st"]

In [154]:
# print key stats (mean + std.dev) for the Kickr and Neo
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
print(f"Wahoo Kickr statistics:")
print(df.loc[df["group"] == "wahoo"].describe())
print(f"\n\n Tacx Neo statistics:")
print(df.loc[df["group"] == "neo"].describe())

Wahoo Kickr statistics:
           zwift_id      p300s_pm      p300s_st         delta
count  1.606500e+04  16065.000000  16065.000000  16065.000000
mean   4.039386e+05    342.054653    340.780517      1.274136
std    3.644162e+05     64.909751     64.688181      9.958776
min    4.990000e+02     59.000000     57.000000    -41.000000
25%    1.038790e+05    304.000000    303.000000     -5.000000
50%    2.912380e+05    349.000000    348.000000      1.000000
75%    6.066520e+05    386.000000    385.000000      7.000000
max    1.545123e+06    529.000000    538.000000     48.000000


 Tacx Neo statistics:
           zwift_id     p300s_pm     p300s_st        delta
count  3.053000e+03  3053.000000  3053.000000  3053.000000
mean   2.918020e+05   329.387815   326.174910     3.212905
std    3.171402e+05    55.982441    55.930461     9.227282
min    3.200000e+02   109.000000   112.000000   -35.000000
25%    7.031400e+04   297.000000   293.000000    -2.000000
50%    1.659770e+05   332.000000   330.0

In [149]:
# export to csv without rider names
df.drop("name", axis = 1).to_csv("data2.csv")